# The Lennard-Jones Fluid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

from matplotlib.animation import FuncAnimation

# from numba import njit # Import this if you're using @njit to speed up your code
from tqdm import tqdm    # This is a very cool package that allows you to see a "progress bar" to know how far a loop has progressed

In [ ]:
# @njit                                        # Once you've written your code using only Numpy arrays, you can choose to speed it up using this command
def lj_acc(rij):
    '''
    Get the 2D Lennard-Jones acceleration of two masses seperated by a distance rij.
    
    Parameters:
    -----------
    • rij : A numpy array of 2 elements storing the `x` and `y` values of the separation `rij`.
    
    Returns:
    --------
    The function returns one value:
    • a : A numpy array 2 elements storing the `x` and `y` components of the acceleration.
    '''

    a=0 
    
    # <===== Q 3(b) Fill this in
    
    return a

# @njit                                        # Once you've written your code using only Numpy arrays, you can choose to speed it up using this command
def get_pe(step_pos,L):
    '''
    Get the 2D Lennard-Jones potential energy for a system of `N` masses.
    
    Parameters:
    -----------
    • pos : An `(N x 2)` numpy array of 2 elements storing the `x` and `y` values of each particle.
    
    Returns:
    --------
    The function returns one value:
    • potential : The combined potential energy of the `N` masses.
    '''
    
    potential = 0

    # <===== Q 3(b) Fill this in
    
    return potential

# @njit                                        # Once you've written your code using only Numpy arrays, you can choose to speed it up using this command
def get_ke(step_vel):
    '''
    Get the kinetic energy for a system of `N` masses.
    
    Parameters:
    -----------
    • vel : An `(N x 2)` numpy array of 2 elements storing the `v_x` and `v_y` values of each particle.
    
    Returns:
    --------
    The function returns one value:
    • kinetic : The combined kinetic energy of the `N` masses.
    '''

    kinetic = 0

    # <===== Q 3(b) Fill this in
    
    return kinetic


# @njit                                        # Once you've written your code using only Numpy arrays, you can choose to speed it up using this command
def pbc_distance(rij):
    '''
    Get the correct distance between two particles, accounting for periodic boundary conditions.
    
    Parameters:
    -----------
    • rij : A numpy array of 2 elements storing the `x` and `y` values of the separation `rij`.
    
    Returns:
    --------
    The function returns one value:
    • rij : The corrected r_ij, including the effects of periodic boundary conditions.
    '''
    
    # <===== Fill this in to compute the _true_ distance, taking into account
    # periodic boundary conditions
    
    return rij

# @njit                                        # Once you've written your code using only Numpy arrays, you can choose to speed it up using this command
def maxwell(v,T):
    '''
    Maxwell-Boltzmann distribution of velocities in two-dimensions.
    
    Parameters:
    -----------
    • v    : Number or array of velocities.
    • T    : Temperature at which the Maxwell-Boltzmann distribution is computed.
    
    Returns:
    --------
    The function returns the probability density from the Maxwell-Boltzmann distribution:
    • p : If `v` is a numpy array, `p` is a numpy array. If `v` is a number, `p` is a number.
    '''

    p = (1./T)*v*np.exp(-0.5*v*v/T)
    
    return p

In [ ]:
def init_pos(Lx, Ly, Nx, Ny):
    '''
    Assign the initial positions of the particles on a regular lattice.
    
    Parameters:
    -----------
    • Lx   : x-length of the lattice.
    • Ly   : y-length of the lattice.
    • Nx   : An integer number of particles in the x-direction.
    • Ny   : An integer number of particles in the y-direction.
    
    Returns:
    --------
    The function returns one value:
    • ipos : A numpy array of (N x 2) elements storing the `x` and `y` coordinates of `N = Nx * Ny` particles.
    '''

    N = Nx*Ny                                       # Total number of particles in the lattice
    
    dx = Lx/Nx                                      # Spacing between particles along the  
    dy = Ly/Ny                                      # x and y axes
    
    ipos = np.zeros((N,2))                          # Empty array to store positions

    n=0                                             # Counter to count the number of particles
    
    for x in range(Nx):                             # Loop over all particles
        for y in range(Ny):
            ipos[n] = [dx/2 + dx*x, dy/2 + dy*y]    # Assign positions to each particle
            n+=1                                    # Increment counter
    
    return ipos


def init_vel(N):
    '''
    Assign the initial velocities of the particles.
    
    Parameters:
    -----------
    • N    : An integer number of particles.
    
    Returns:
    --------
    The function returns one value:
    • ivel : A numpy array of (N x 2) elements storing the `x` and `y` velocity components of `N` particles.
    '''
    
    ivel = np.zeros((N,2), dtype=np.float32)         # Initialise the empty array for velocities 

    T = 2.5                                          # Initial "temperature" of the system
    T0 = 1                                           # Temperature scale of the problem

    v0 = np.sqrt(2*T/T0)                             # Initial speed of each particle

    for i in range(N):                               # Loop over each particle and 
        r = 2*np.pi*np.random.random()               # assign it a random direction
        ivel[i][0] = v0 * np.cos(r)                  # Set the x and y components of the velocity
        ivel[i][1] = v0 * np.sin(r)                  # while keeping the magnitude constant

    vcm = np.sum(ivel,axis=0)/N                      # Compute the centre of mass velocity
    ivel = ivel-vcm                                  # Subtract the COM velocity so the COM is stationary

    return ivel

In [ ]:
# @njit                                        # Once you've written your code using only Numpy arrays, you can choose to speed it up using this command
def simulate(ipos, ivel, tf=100, dt=0.01):
    '''
    Run a simulation given an initial position and velocity array.
    
    Parameters:
    -----------
    • ipos : An (Nx2) array of initial positions of N particles in 2D.
    • ivel : An (Nx2) array of initial velocities of N particles in 2D.
    • tf   : (optional) Final time until which the simulation will run.
    • dt   : (optional) Time-step between simulation steps.
    
    Returns:
    --------
    The function four values
    • pos  : An array of `(n_steps x N x 2)` elements containing the (x,y) coordinates
             of each particle at each time-step. Eg: `pos[10]` gives a snapshot of the system
             at time-step 10.
    • vel  : An array of `(n_steps x N x 2)` elements containing the (v_x,v_y) coordinates
             of each particle at each time-step. Eg: `vel[10]` gives the velocities of the system
             at time-step 10.
    • KE   : An array of `n_steps` elements containing the total kinetic energy of the system at each step.
    • PE   : An array of `n_steps` elements containing the total potential energy of the system at each step.
    '''
    
    n_steps = int(tf/dt)                                     # Number of simulation steps

    pos = np.zeros(shape=(n_steps,N,2), dtype=np.float32)    # Arrays to store the history of the particles'
    vel = np.zeros(shape=(n_steps,N,2), dtype=np.float32)    # positions and velocities

    KE = np.zeros(n_steps)                                   # Arrays to store the history of the particles'
    PE = np.zeros(n_steps)                                   # kinetic and potential energies

    pos[0] = ipos                                            # Setting initial positions to `ipos`
    vel[0] = ivel                                            # Setting initial velocities to `ivel`

    KE[0] = get_ke(vel[0])                                   # Computing initial kinetic and potential
    PE[0] = get_pe(pos[0], L=L)                              # energies

    for s in tqdm(range(1,n_steps)):                         # Loop over steps. NOTE: tqdm doesn't work with Numba
        
        # <===== Q3(a) Fill this in: Increment positions to value at intermediate point

        # <===== Q3(a) Impose periodic boundary conditions for each dimension (x and y)
        

        net_a = np.zeros((N,2), dtype=np.float32)            # Empty array to store the net acceleration of each particle

        for p in range(N-1):                                 # Computing the acceleration for each _pair_ of particles              
            for q in range(p+1,N):
                
                acc = 0 # acc = <==== Compute the acceleration between pairs of particles
                 
                net_a[p] += acc                              # Use the third-law to impart equal and opposite 
                net_a[q] -= acc                              # accelerations to each particle


        # <===== Q3(a) Update the velocity and final positions using the Verlet scheme

        # <===== Q3(a) Re-impose periodic boundary conditions for each dimension (x and y)

        KE[s] = get_ke(vel[s])                               # Compute the kinetic and potential energies at 
        PE[s] = get_pe(pos[s], L=L)                          # the current step

    
    return pos, vel, KE, PE
        

In [ ]:
##############################################
########### SIMULATION PARAMETERS ############
##############################################

Nx = 5                   # Number of particles per row initially along x
Ny = 5                   # Number of particles per row iniailly along y
N = Nx*Ny                # Total number of particles
rho = 0.5                # Density of the system
L = np.sqrt(N/rho)       # Length of the system

ipos = init_pos(Lx=L, Ly=L, Nx=Nx, Ny=Ny)  # Initial positions (square grid)
ivel = init_vel(N)                         # Initial velocities 

tf = 100                                   # Total time duration
dt = 0.01                                  # Time-step
time = np.arange(0,int(tf/dt))*dt          # Array of all time-steps

In [ ]:
%%time
##############################################
##### Run a single simulation forwards #######
##############################################

# For N=25, tf=100, and dt=0.01, this should take 
# about ~ 30 s to run without Numba, and ~ 1 s with Numba

pos,vel, KE, PE = simulate(ipos, ivel, tf=tf, dt=dt)

In [ ]:
%%time
##############################################
##### Run the same simulation backwards ######
##############################################

# For N=25, tf=100, and dt=0.01, this should take 
# about ~ 30 s to run without Numba, and ~ 1 s with Numba

# Q3(c) <==== Fill this in to simulate the system _backwards_ starting from the last configuration of the previous cell

In [ ]:
# This function has been written for you, you don't need to modify it unless you want to.
def animate_particles(pos, save=False, save_name="lj.mp4", save_dpi=100, save_fps=100):
    '''
    Animate a set of step-wise positions of `N` particles, with the option of saving a movie.
    
    Parameters:
    -----------
    • pos       : An array of `(n_steps x N x 2)` elements representing a snapshot of the system at each step.
    • save      : (Optional) Boolean to save the animation as a movie.
    • save_name : (Optional) Name of the output video.
    • save_dpi  : (Optional) DPI of the output movie.
    • save_fps  : (Optional) Frames-per-second of the output movie.
    
    Returns:
    --------
    This function returns an animation
    • ani       : A FuncAnimation object.
    '''
    
    fig, ax = plt.subplots()      # Create an empty plot and fill it with the initial position of the particles
    plot, = ax.plot(pos[0,:,0], pos[0,:,1], ls='none',color='firebrick', marker='o', mec='darkred', ms=10)

    def plotsetup():              # Function to setup the plot canvas
        ax.set_aspect('equal')
        ax.set_xlim(0,L)
        ax.set_ylim(0,L)
    
    def animate(frame):           # `Animate` function called by the FuncAnimation object below at each frame
        plot.set_data(pos[frame,:,0], pos[frame,:,1])

        return plot,
    
    steps = len(pos)              # Total steps
    
    ani = FuncAnimation(fig, animate, init_func=plotsetup(), frames=range(0,steps,int(1/dt)), interval=50, blit=True, repeat=True)

    if(save): # If `save==True`, a movie is created
        ani.save(save_name, dpi=save_dpi, fps=save_fps)
    
    return ani
    

In [ ]:
%matplotlib notebook  
# Animations require some sort of matplotlib backend, either %matplotlib notebook, or matplotlib tk
animate_particles(pos)

In [ ]:
# Q3(d) <==== Plot the total, kinetic, and potential energies

In [ ]:
# Q3(d) <==== Show that the cumulative values of these energies stabilises after some "equilibration" time

In [ ]:
# Q3(d) <==== Plot a histogram of all the speeds that occur after equilibration has been attained, over multiple steps.
#             Note: The `density=True` parameter of plt.hist will be useful to plot this histogram since itis essentially
#                   a probability density.